### 개선판 word2vec 학습

### CBOW 모델 구현

In [2]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None


In [2]:
# coding: utf-8
import sys
sys.path.append('..')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.layers import *
from ch04.negative_sampling_layer import NegativeSamplingLoss


class SkipGram:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        rn = np.random.randn

        # 가중치 초기화
        W_in = 0.01 * rn(V, H).astype('f')
        W_out = 0.01 * rn(V, H).astype('f')

        # 계층 생성
        self.in_layer = Embedding(W_in)
        self.loss_layers = []
        for i in range(2 * window_size):
            layer = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
            self.loss_layers.append(layer)

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer] + self.loss_layers
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = self.in_layer.forward(target)

        loss = 0
        for i, layer in enumerate(self.loss_layers):
            loss += layer.forward(h, contexts[:, i])
        return loss

    def backward(self, dout=1):
        dh = 0
        for i, layer in enumerate(self.loss_layers):
            dh += layer.backward(dout)
        self.in_layer.backward(dh)
        return None


### CBOW 모델 학습 코드

In [3]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 1[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 2[s] | 손실 4.13
| 에폭 1 |  반복 81 / 9295 | 시간 2[s] | 손실 4.05
| 에폭 1 |  반복 101 / 9295 | 시간 3[s] | 손실 3.92
| 에폭 1 |  반복 121 / 9295 | 시간 4[s] | 손실 3.78
| 에폭 1 |  반복 141 / 9295 | 시간 4[s] | 손실 3.63
| 에폭 1 |  반복 161 / 9295 | 시간 5[s] | 손실 3.48
| 에폭 1 |  반복 181 / 9295 | 시간 6[s] | 손실 3.37
| 에폭 1 |  반복 201 / 9295 | 시간 6[s] | 손실 3.26
| 에폭 1 |  반복 221 / 9295 | 시간 7[s] | 손실 3.16
| 에폭 1 |  반복 241 / 9295 | 시간 8[s] | 손실 3.10
| 에폭 1 |  반복 261 / 9295 | 시간 8[s] | 손실 3.03
| 에폭 1 |  반복 281 / 9295 | 시간 9[s] | 손실 2.96
| 에폭 1 |  반복 301 / 9295 | 시간 10[s] | 손실 2.90
| 에폭 1 |  반복 321 / 9295 | 시간 10[s] | 손실 2.89
| 에폭 1 |  반복 341 / 9295 | 시간 11[s] | 손실 2.85
| 에폭 1 |  반복 361 / 9295 | 시간 12[s] | 손실 2.81
| 에폭 1 |  반복 381 / 9295 | 시간 12[s] | 손실 2.79
| 에폭 1 |  반복 401 / 9295 | 시간 13[s] | 손실 2.78
| 에폭 1 |  반복 421 / 9295 | 시간 14[s] | 손실 2.74
| 에폭 1 |  반복 441 / 9295 | 시간 14

| 에폭 1 |  반복 3601 / 9295 | 시간 118[s] | 손실 2.41
| 에폭 1 |  반복 3621 / 9295 | 시간 119[s] | 손실 2.39
| 에폭 1 |  반복 3641 / 9295 | 시간 120[s] | 손실 2.36
| 에폭 1 |  반복 3661 / 9295 | 시간 120[s] | 손실 2.43
| 에폭 1 |  반복 3681 / 9295 | 시간 121[s] | 손실 2.41
| 에폭 1 |  반복 3701 / 9295 | 시간 122[s] | 손실 2.40
| 에폭 1 |  반복 3721 / 9295 | 시간 123[s] | 손실 2.40
| 에폭 1 |  반복 3741 / 9295 | 시간 123[s] | 손실 2.44
| 에폭 1 |  반복 3761 / 9295 | 시간 124[s] | 손실 2.45
| 에폭 1 |  반복 3781 / 9295 | 시간 125[s] | 손실 2.43
| 에폭 1 |  반복 3801 / 9295 | 시간 125[s] | 손실 2.41
| 에폭 1 |  반복 3821 / 9295 | 시간 126[s] | 손실 2.38
| 에폭 1 |  반복 3841 / 9295 | 시간 127[s] | 손실 2.39
| 에폭 1 |  반복 3861 / 9295 | 시간 127[s] | 손실 2.42
| 에폭 1 |  반복 3881 / 9295 | 시간 128[s] | 손실 2.40
| 에폭 1 |  반복 3901 / 9295 | 시간 129[s] | 손실 2.41
| 에폭 1 |  반복 3921 / 9295 | 시간 129[s] | 손실 2.39
| 에폭 1 |  반복 3941 / 9295 | 시간 130[s] | 손실 2.39
| 에폭 1 |  반복 3961 / 9295 | 시간 130[s] | 손실 2.39
| 에폭 1 |  반복 3981 / 9295 | 시간 131[s] | 손실 2.41
| 에폭 1 |  반복 4001 / 9295 | 시간 132[s] | 손실 2.43
| 에폭 1 |  반복 

| 에폭 1 |  반복 7101 / 9295 | 시간 235[s] | 손실 2.31
| 에폭 1 |  반복 7121 / 9295 | 시간 235[s] | 손실 2.28
| 에폭 1 |  반복 7141 / 9295 | 시간 236[s] | 손실 2.30
| 에폭 1 |  반복 7161 / 9295 | 시간 237[s] | 손실 2.30
| 에폭 1 |  반복 7181 / 9295 | 시간 237[s] | 손실 2.28
| 에폭 1 |  반복 7201 / 9295 | 시간 238[s] | 손실 2.27
| 에폭 1 |  반복 7221 / 9295 | 시간 239[s] | 손실 2.26
| 에폭 1 |  반복 7241 / 9295 | 시간 239[s] | 손실 2.28
| 에폭 1 |  반복 7261 / 9295 | 시간 240[s] | 손실 2.27
| 에폭 1 |  반복 7281 / 9295 | 시간 241[s] | 손실 2.27
| 에폭 1 |  반복 7301 / 9295 | 시간 241[s] | 손실 2.28
| 에폭 1 |  반복 7321 / 9295 | 시간 242[s] | 손실 2.30
| 에폭 1 |  반복 7341 / 9295 | 시간 243[s] | 손실 2.30
| 에폭 1 |  반복 7361 / 9295 | 시간 243[s] | 손실 2.29
| 에폭 1 |  반복 7381 / 9295 | 시간 244[s] | 손실 2.30
| 에폭 1 |  반복 7401 / 9295 | 시간 245[s] | 손실 2.28
| 에폭 1 |  반복 7421 / 9295 | 시간 245[s] | 손실 2.25
| 에폭 1 |  반복 7441 / 9295 | 시간 246[s] | 손실 2.26
| 에폭 1 |  반복 7461 / 9295 | 시간 247[s] | 손실 2.26
| 에폭 1 |  반복 7481 / 9295 | 시간 247[s] | 손실 2.25
| 에폭 1 |  반복 7501 / 9295 | 시간 248[s] | 손실 2.26
| 에폭 1 |  반복 

| 에폭 2 |  반복 1321 / 9295 | 시간 649[s] | 손실 2.13
| 에폭 2 |  반복 1341 / 9295 | 시간 650[s] | 손실 2.14
| 에폭 2 |  반복 1361 / 9295 | 시간 650[s] | 손실 2.18
| 에폭 2 |  반복 1381 / 9295 | 시간 651[s] | 손실 2.15
| 에폭 2 |  반복 1401 / 9295 | 시간 652[s] | 손실 2.12
| 에폭 2 |  반복 1421 / 9295 | 시간 652[s] | 손실 2.15
| 에폭 2 |  반복 1441 / 9295 | 시간 653[s] | 손실 2.16
| 에폭 2 |  반복 1461 / 9295 | 시간 654[s] | 손실 2.16
| 에폭 2 |  반복 1481 / 9295 | 시간 654[s] | 손실 2.15
| 에폭 2 |  반복 1501 / 9295 | 시간 655[s] | 손실 2.17
| 에폭 2 |  반복 1521 / 9295 | 시간 656[s] | 손실 2.13
| 에폭 2 |  반복 1541 / 9295 | 시간 656[s] | 손실 2.17
| 에폭 2 |  반복 1561 / 9295 | 시간 657[s] | 손실 2.17
| 에폭 2 |  반복 1581 / 9295 | 시간 658[s] | 손실 2.16
| 에폭 2 |  반복 1601 / 9295 | 시간 658[s] | 손실 2.13
| 에폭 2 |  반복 1621 / 9295 | 시간 659[s] | 손실 2.15
| 에폭 2 |  반복 1641 / 9295 | 시간 660[s] | 손실 2.17
| 에폭 2 |  반복 1661 / 9295 | 시간 660[s] | 손실 2.14
| 에폭 2 |  반복 1681 / 9295 | 시간 661[s] | 손실 2.12
| 에폭 2 |  반복 1701 / 9295 | 시간 662[s] | 손실 2.17
| 에폭 2 |  반복 1721 / 9295 | 시간 662[s] | 손실 2.14
| 에폭 2 |  반복 

| 에폭 2 |  반복 4821 / 9295 | 시간 765[s] | 손실 2.12
| 에폭 2 |  반복 4841 / 9295 | 시간 765[s] | 손실 2.10
| 에폭 2 |  반복 4861 / 9295 | 시간 766[s] | 손실 2.08
| 에폭 2 |  반복 4881 / 9295 | 시간 767[s] | 손실 2.12
| 에폭 2 |  반복 4901 / 9295 | 시간 767[s] | 손실 2.06
| 에폭 2 |  반복 4921 / 9295 | 시간 768[s] | 손실 2.13
| 에폭 2 |  반복 4941 / 9295 | 시간 769[s] | 손실 2.09
| 에폭 2 |  반복 4961 / 9295 | 시간 769[s] | 손실 2.08
| 에폭 2 |  반복 4981 / 9295 | 시간 770[s] | 손실 2.08
| 에폭 2 |  반복 5001 / 9295 | 시간 771[s] | 손실 2.10
| 에폭 2 |  반복 5021 / 9295 | 시간 771[s] | 손실 2.06
| 에폭 2 |  반복 5041 / 9295 | 시간 772[s] | 손실 2.07
| 에폭 2 |  반복 5061 / 9295 | 시간 773[s] | 손실 2.07
| 에폭 2 |  반복 5081 / 9295 | 시간 773[s] | 손실 2.04
| 에폭 2 |  반복 5101 / 9295 | 시간 774[s] | 손실 2.07
| 에폭 2 |  반복 5121 / 9295 | 시간 775[s] | 손실 2.10
| 에폭 2 |  반복 5141 / 9295 | 시간 775[s] | 손실 2.08
| 에폭 2 |  반복 5161 / 9295 | 시간 776[s] | 손실 2.09
| 에폭 2 |  반복 5181 / 9295 | 시간 777[s] | 손실 2.05
| 에폭 2 |  반복 5201 / 9295 | 시간 777[s] | 손실 2.09
| 에폭 2 |  반복 5221 / 9295 | 시간 778[s] | 손실 2.09
| 에폭 2 |  반복 

| 에폭 2 |  반복 8321 / 9295 | 시간 881[s] | 손실 2.01
| 에폭 2 |  반복 8341 / 9295 | 시간 881[s] | 손실 2.04
| 에폭 2 |  반복 8361 / 9295 | 시간 882[s] | 손실 2.04
| 에폭 2 |  반복 8381 / 9295 | 시간 883[s] | 손실 2.05
| 에폭 2 |  반복 8401 / 9295 | 시간 883[s] | 손실 2.03
| 에폭 2 |  반복 8421 / 9295 | 시간 884[s] | 손실 1.96
| 에폭 2 |  반복 8441 / 9295 | 시간 885[s] | 손실 2.03
| 에폭 2 |  반복 8461 / 9295 | 시간 885[s] | 손실 2.04
| 에폭 2 |  반복 8481 / 9295 | 시간 886[s] | 손실 2.01
| 에폭 2 |  반복 8501 / 9295 | 시간 887[s] | 손실 2.03
| 에폭 2 |  반복 8521 / 9295 | 시간 887[s] | 손실 2.04
| 에폭 2 |  반복 8541 / 9295 | 시간 888[s] | 손실 2.08
| 에폭 2 |  반복 8561 / 9295 | 시간 889[s] | 손실 2.02
| 에폭 2 |  반복 8581 / 9295 | 시간 889[s] | 손실 2.02
| 에폭 2 |  반복 8601 / 9295 | 시간 890[s] | 손실 1.99
| 에폭 2 |  반복 8621 / 9295 | 시간 891[s] | 손실 2.02
| 에폭 2 |  반복 8641 / 9295 | 시간 891[s] | 손실 2.03
| 에폭 2 |  반복 8661 / 9295 | 시간 892[s] | 손실 2.03
| 에폭 2 |  반복 8681 / 9295 | 시간 893[s] | 손실 2.01
| 에폭 2 |  반복 8701 / 9295 | 시간 893[s] | 손실 2.02
| 에폭 2 |  반복 8721 / 9295 | 시간 894[s] | 손실 1.99
| 에폭 2 |  반복 

| 에폭 3 |  반복 2541 / 9295 | 시간 998[s] | 손실 1.94
| 에폭 3 |  반복 2561 / 9295 | 시간 998[s] | 손실 1.97
| 에폭 3 |  반복 2581 / 9295 | 시간 999[s] | 손실 1.92
| 에폭 3 |  반복 2601 / 9295 | 시간 1000[s] | 손실 1.90
| 에폭 3 |  반복 2621 / 9295 | 시간 1000[s] | 손실 1.93
| 에폭 3 |  반복 2641 / 9295 | 시간 1001[s] | 손실 1.94
| 에폭 3 |  반복 2661 / 9295 | 시간 1002[s] | 손실 1.97
| 에폭 3 |  반복 2681 / 9295 | 시간 1002[s] | 손실 1.93
| 에폭 3 |  반복 2701 / 9295 | 시간 1003[s] | 손실 1.92
| 에폭 3 |  반복 2721 / 9295 | 시간 1004[s] | 손실 1.93
| 에폭 3 |  반복 2741 / 9295 | 시간 1004[s] | 손실 1.95
| 에폭 3 |  반복 2761 / 9295 | 시간 1005[s] | 손실 1.96
| 에폭 3 |  반복 2781 / 9295 | 시간 1006[s] | 손실 1.93
| 에폭 3 |  반복 2801 / 9295 | 시간 1006[s] | 손실 1.97
| 에폭 3 |  반복 2821 / 9295 | 시간 1007[s] | 손실 1.91
| 에폭 3 |  반복 2841 / 9295 | 시간 1008[s] | 손실 1.93
| 에폭 3 |  반복 2861 / 9295 | 시간 1008[s] | 손실 1.94
| 에폭 3 |  반복 2881 / 9295 | 시간 1009[s] | 손실 1.93
| 에폭 3 |  반복 2901 / 9295 | 시간 1010[s] | 손실 1.96
| 에폭 3 |  반복 2921 / 9295 | 시간 1010[s] | 손실 1.98
| 에폭 3 |  반복 2941 / 9295 | 시간 1011[s] | 손실 

| 에폭 3 |  반복 5961 / 9295 | 시간 1291[s] | 손실 1.91
| 에폭 3 |  반복 5981 / 9295 | 시간 1292[s] | 손실 1.92
| 에폭 3 |  반복 6001 / 9295 | 시간 1293[s] | 손실 1.92
| 에폭 3 |  반복 6021 / 9295 | 시간 1293[s] | 손실 1.91
| 에폭 3 |  반복 6041 / 9295 | 시간 1294[s] | 손실 1.91
| 에폭 3 |  반복 6061 / 9295 | 시간 1295[s] | 손실 1.91
| 에폭 3 |  반복 6081 / 9295 | 시간 1295[s] | 손실 1.91
| 에폭 3 |  반복 6101 / 9295 | 시간 1296[s] | 손실 1.94
| 에폭 3 |  반복 6121 / 9295 | 시간 1297[s] | 손실 1.89
| 에폭 3 |  반복 6141 / 9295 | 시간 1297[s] | 손실 1.94
| 에폭 3 |  반복 6161 / 9295 | 시간 1298[s] | 손실 1.86
| 에폭 3 |  반복 6181 / 9295 | 시간 1299[s] | 손실 1.92
| 에폭 3 |  반복 6201 / 9295 | 시간 1299[s] | 손실 1.91
| 에폭 3 |  반복 6221 / 9295 | 시간 1300[s] | 손실 1.89
| 에폭 3 |  반복 6241 / 9295 | 시간 1301[s] | 손실 1.89
| 에폭 3 |  반복 6261 / 9295 | 시간 1301[s] | 손실 1.94
| 에폭 3 |  반복 6281 / 9295 | 시간 1302[s] | 손실 1.89
| 에폭 3 |  반복 6301 / 9295 | 시간 1303[s] | 손실 1.90
| 에폭 3 |  반복 6321 / 9295 | 시간 1303[s] | 손실 1.93
| 에폭 3 |  반복 6341 / 9295 | 시간 1304[s] | 손실 1.88
| 에폭 3 |  반복 6361 / 9295 | 시간 1305[s] | 

KeyboardInterrupt: 

###  CBOW 모델 평가

In [4]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle

# 훈련된 파일
pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.71533203125
 i: 0.705078125
 your: 0.626953125
 they: 0.6171875
 anything: 0.59716796875

[query] year
 month: 0.83837890625
 week: 0.787109375
 spring: 0.767578125
 summer: 0.759765625
 decade: 0.697265625

[query] car
 truck: 0.6337890625
 luxury: 0.6328125
 auto: 0.5751953125
 window: 0.57421875
 cars: 0.55859375

[query] toyota
 honda: 0.658203125
 marathon: 0.6279296875
 coated: 0.62646484375
 engines: 0.61962890625
 seita: 0.61669921875


In [5]:
# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)

--------------------------------------------------

[analogy] king:man = queen:?
 a.m: 5.7421875
 woman: 5.50390625
 naczelnik: 5.10546875
 carolinas: 4.93359375
 hacker: 4.74609375

[analogy] take:took = go:?
 went: 4.6796875
 're: 4.45703125
 came: 4.4140625
 were: 4.07421875
 was: 4.0625

[analogy] car:cars = child:?
 a.m: 6.75390625
 rape: 5.875
 children: 5.3203125
 daffynition: 4.734375
 women: 4.5390625

[analogy] good:better = bad:?
 rather: 5.78125
 more: 5.73828125
 less: 5.40625
 greater: 4.5078125
 fewer: 4.1328125
